# Predict Future Sales: feature engineering and ensemble approach

# Exploratory Data Analysis

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

from itertools import product
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import time
import sys
import gc
import pickle
sys.version_info

In [ ]:
# load data
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
cats = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
train = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
# set index to ID to avoid droping it later
test  = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv').set_index('ID')

In [ ]:
train.head()

Test set is a product of some shops and some items within 34 month. There are 5100 items * 42 shops = 214200 pairs. 363 items are new compared to the train. 

In [ ]:
print('train size, item in train, shop in train', train.shape[0], train.item_id.nunique(), train.shop_id.nunique())
print('train size, item in train, shop in train', test.shape[0], test.item_id.nunique(),test.shop_id.nunique())
print('new items:', len(list(set(test.item_id) - set(test.item_id).intersection(set(train.item_id)))), len(list(set(test.item_id))), len(test))

Check for missing values.

In [ ]:
train.isnull().sum()

There is no missing value in the training set.

Plot the total sale of each month, we see a clear trend and seasonality. The overall sale is decreasing with time, and there are peaks in November.

In [ ]:
sale_by_month = train.groupby('date_block_num')['item_cnt_day'].sum()
sale_by_month.plot()

The distribution of sale grouped by month, item and shop, we see most item-shop pairs have small monthly sale.

In [ ]:
block_item_shop_sale = train.groupby(['date_block_num','item_id','shop_id'])['item_cnt_day'].sum()
block_item_shop_sale.clip(0,20).plot.hist(bins=20)


# Feature engineering and data cleaning

#### remove outliers
Remove outliers with very large item_cnt_day and item_price.

In [ ]:
plt.figure(figsize=(10,4))
plt.xlim(-100, 3000)
sns.boxplot(x=train.item_cnt_day)

plt.figure(figsize=(10,4))
plt.xlim(train.item_price.min(), train.item_price.max()*1.1)
sns.boxplot(x=train.item_price)

train = train[train.item_price<100000]
train = train[train.item_cnt_day<1001]

There is one item with price below zero. Fill it with median.

In [ ]:
median = train[(train.shop_id==32)&(train.item_id==2973)&(train.date_block_num==4)&(train.item_price>0)].item_price.median()
train.loc[train.item_price<0, 'item_price'] = median

Several shops are duplicates of each other (according to its name). Fix train and test set.

In [ ]:
# Якутск Орджоникидзе, 56
train.loc[train.shop_id == 0, 'shop_id'] = 57
test.loc[test.shop_id == 0, 'shop_id'] = 57
# Якутск ТЦ "Центральный"
train.loc[train.shop_id == 1, 'shop_id'] = 58
test.loc[test.shop_id == 1, 'shop_id'] = 58
# Жуковский ул. Чкалова 39м²
train.loc[train.shop_id == 10, 'shop_id'] = 11
test.loc[test.shop_id == 10, 'shop_id'] = 11

#### Shops/Cats/Items preprocessing
Observations:
* Each shop_name starts with the city name.
* Each category contains type and subtype in its name.

In [ ]:
shops.loc[shops.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
shops['city'] = shops['shop_name'].str.split(' ').map(lambda x: x[0])
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'
shops['city_code'] = LabelEncoder().fit_transform(shops['city'])
shops = shops[['shop_id','city_code']]

cats['split'] = cats['item_category_name'].str.split('-')
cats['type'] = cats['split'].map(lambda x: x[0].strip())
cats['type_code'] = LabelEncoder().fit_transform(cats['type'])
# if subtype is nan then type
cats['subtype'] = cats['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
cats['subtype_code'] = LabelEncoder().fit_transform(cats['subtype'])
cats = cats[['item_category_id','type_code', 'subtype_code']]

items.drop(['item_name'], axis=1, inplace=True)

## Monthly sales
Most of the items in the test set target value should be zero, while train set contains only pairs which were sold or returned in the past. So we expand the train set to include those item-shop pairs with zero monthly sales. This way train data will be similar to test data.

In [ ]:
ts = time.time()
matrix = []
cols = ['date_block_num','shop_id','item_id']
for i in range(34):
    sales = train[train.date_block_num==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))
    
matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix['item_id'] = matrix['item_id'].astype(np.int16)
matrix.sort_values(cols,inplace=True)
time.time() - ts

Aggregate train set by shop/item pairs to calculate target aggreagates, then <b>clip(0,20)</b> target value. This way train target will be similar to the test predictions.

Downcast item_cnt_month to float32 -- float16 was too small to perform sum operation.

In [ ]:
train['revenue'] = train['item_price'] *  train['item_cnt_day']

In [ ]:

group = train.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': ['sum']})
group.columns = ['item_cnt_month']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=cols, how='left')
matrix['item_cnt_month'] = (matrix['item_cnt_month']
                                .fillna(0)
                                .clip(0,20) # NB clip target here
                                .astype(np.float32))


## Test set
To use time tricks append test pairs to the matrix.

In [ ]:
test['date_block_num'] = 34
test['date_block_num'] = test['date_block_num'].astype(np.int8)
test['shop_id'] = test['shop_id'].astype(np.int8)
test['item_id'] = test['item_id'].astype(np.int16)

In [ ]:

matrix = pd.concat([matrix, test], ignore_index=True, sort=False, keys=cols)
matrix.fillna(0, inplace=True) # 34 month


## Shops/Items/Cats features

In [ ]:

matrix = pd.merge(matrix, shops, on=['shop_id'], how='left')
matrix = pd.merge(matrix, items, on=['item_id'], how='left')
matrix = pd.merge(matrix, cats, on=['item_category_id'], how='left')
matrix['city_code'] = matrix['city_code'].astype(np.int8)
matrix['item_category_id'] = matrix['item_category_id'].astype(np.int8)
matrix['type_code'] = matrix['type_code'].astype(np.int8)
matrix['subtype_code'] = matrix['subtype_code'].astype(np.int8)


## Target lags

In [ ]:
def lag_feature(df, lags, col):
    tmp = df[['date_block_num','shop_id','item_id',col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num','shop_id','item_id', col+'_lag_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num','shop_id','item_id'], how='left')
    return df

In [ ]:

matrix = lag_feature(matrix, [1,2,3,6,12], 'item_cnt_month')

## Group sale stats in recent
create stats (mean/var) of sales of certain groups during the past 12 months

In [ ]:
def add_group_stats(matrix_, groupby_feats, target, enc_feat, last_periods):
    if not 'date_block_num' in groupby_feats:
        print ('date_block_num must in groupby_feats')
        return matrix_
    
    group = matrix_.groupby(groupby_feats)[target].sum().reset_index()
    max_lags = np.max(last_periods)
    for i in range(1,max_lags+1):
        shifted = group[groupby_feats+[target]].copy(deep=True)
        shifted['date_block_num'] += i
        shifted.rename({target:target+'_lag_'+str(i)},axis=1,inplace=True)
        group = group.merge(shifted, on=groupby_feats, how='left')
    group.fillna(0,inplace=True)
    for period in last_periods:
        lag_feats = [target+'_lag_'+str(lag) for lag in np.arange(1,period+1)]
        # we do not use mean and svd directly because we want to include months with sales = 0
        mean = group[lag_feats].sum(axis=1)/float(period)
        mean2 = (group[lag_feats]**2).sum(axis=1)/float(period)
        group[enc_feat+'_avg_sale_last_'+str(period)] = mean
        group[enc_feat+'_std_sale_last_'+str(period)] = (mean2 - mean**2).apply(np.sqrt)
        group[enc_feat+'_std_sale_last_'+str(period)].replace(np.inf,0,inplace=True)
        # divide by mean, this scales the features for NN
        group[enc_feat+'_avg_sale_last_'+str(period)] /= group[enc_feat+'_avg_sale_last_'+str(period)].mean()
        group[enc_feat+'_std_sale_last_'+str(period)] /= group[enc_feat+'_std_sale_last_'+str(period)].mean()
    cols = groupby_feats + [f_ for f_ in group.columns.values if f_.find('_sale_last_')>=0]
    matrix = matrix_.merge(group[cols], on=groupby_feats, how='left')
    return matrix

In [ ]:
ts = time.time()
matrix = add_group_stats(matrix, ['date_block_num', 'item_id'], 'item_cnt_month', 'item', [6,12])
matrix = add_group_stats(matrix, ['date_block_num', 'shop_id'], 'item_cnt_month', 'shop', [6,12])
matrix = add_group_stats(matrix, ['date_block_num', 'item_category_id'], 'item_cnt_month', 'category', [12])
matrix = add_group_stats(matrix, ['date_block_num', 'city_code'], 'item_cnt_month', 'city', [12])
matrix = add_group_stats(matrix, ['date_block_num', 'type_code'], 'item_cnt_month', 'type', [12])
matrix = add_group_stats(matrix, ['date_block_num', 'subtype_code'], 'item_cnt_month', 'subtype', [12])
time.time() - ts

## Lag features

In [ ]:
#first use target encoding each group, then shift month to creat lag features
def target_encoding(matrix_, groupby_feats, target, enc_feat, lags):
    print ('target encoding for',groupby_feats)
    group = matrix_.groupby(groupby_feats).agg({target:'mean'})
    group.columns = [enc_feat]
    group.reset_index(inplace=True)
    matrix = matrix_.merge(group, on=groupby_feats, how='left')
    matrix[enc_feat] = matrix[enc_feat].astype(np.float16)
    matrix = lag_feature(matrix, lags, enc_feat)
    matrix.drop(enc_feat, axis=1, inplace=True)
    return matrix

In [ ]:
ts = time.time()
matrix = target_encoding(matrix, ['date_block_num'], 'item_cnt_month', 'date_avg_item_cnt', [1])
matrix = target_encoding(matrix, ['date_block_num', 'item_id'], 'item_cnt_month', 'date_item_avg_item_cnt', [1,2,3,6,12])
matrix = target_encoding(matrix, ['date_block_num', 'shop_id'], 'item_cnt_month', 'date_shop_avg_item_cnt', [1,2,3,6,12])
matrix = target_encoding(matrix, ['date_block_num', 'item_category_id'], 'item_cnt_month', 'date_cat_avg_item_cnt', [1])
matrix = target_encoding(matrix, ['date_block_num', 'shop_id', 'item_category_id'], 'item_cnt_month', 'date_shop_cat_avg_item_cnt', [1])
matrix = target_encoding(matrix, ['date_block_num', 'city_code'], 'item_cnt_month', 'date_city_avg_item_cnt', [1])
matrix = target_encoding(matrix, ['date_block_num', 'item_id', 'city_code'], 'item_cnt_month', 'date_item_city_avg_item_cnt', [1])
time.time() - ts

## Trend features

Price trend for the last six months.

In [ ]:
ts = time.time()
group = train.groupby(['item_id']).agg({'item_price': ['mean']})
group.columns = ['item_avg_item_price']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['item_id'], how='left')
matrix['item_avg_item_price'] = matrix['item_avg_item_price'].astype(np.float16)

group = train.groupby(['date_block_num','item_id']).agg({'item_price': ['mean']})
group.columns = ['date_item_avg_item_price']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','item_id'], how='left')
matrix['date_item_avg_item_price'] = matrix['date_item_avg_item_price'].astype(np.float16)

lags = [1,2,3,4,5,6]
matrix = lag_feature(matrix, lags, 'date_item_avg_item_price')

for i in lags:
    matrix['delta_price_lag_'+str(i)] = \
        (matrix['date_item_avg_item_price_lag_'+str(i)] - matrix['item_avg_item_price']) / matrix['item_avg_item_price']

def select_trend(row):
    for i in lags:
        if row['delta_price_lag_'+str(i)]:
            return row['delta_price_lag_'+str(i)]
    return 0
    
matrix['delta_price_lag'] = matrix.apply(select_trend, axis=1)
matrix['delta_price_lag'] = matrix['delta_price_lag'].astype(np.float16)
matrix['delta_price_lag'].fillna(0, inplace=True)

fetures_to_drop = ['item_avg_item_price', 'date_item_avg_item_price']
for i in lags:
    fetures_to_drop += ['date_item_avg_item_price_lag_'+str(i)]
    fetures_to_drop += ['delta_price_lag_'+str(i)]

matrix.drop(fetures_to_drop, axis=1, inplace=True)

time.time() - ts

Last month shop revenue trend

In [ ]:
ts = time.time()
group = train.groupby(['date_block_num','shop_id']).agg({'revenue': ['sum']})
group.columns = ['date_shop_revenue']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['date_block_num','shop_id'], how='left')
matrix['date_shop_revenue'] = matrix['date_shop_revenue'].astype(np.float32)

group = group.groupby(['shop_id']).agg({'date_shop_revenue': ['mean']})
group.columns = ['shop_avg_revenue']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=['shop_id'], how='left')
matrix['shop_avg_revenue'] = matrix['shop_avg_revenue'].astype(np.float32)

matrix['delta_revenue'] = (matrix['date_shop_revenue'] - matrix['shop_avg_revenue']) / matrix['shop_avg_revenue']
matrix['delta_revenue'] = matrix['delta_revenue'].astype(np.float16)

matrix = lag_feature(matrix, [1], 'delta_revenue')

matrix.drop(['date_shop_revenue','shop_avg_revenue','delta_revenue'], axis=1, inplace=True)
time.time() - ts

## Add month and year

In [ ]:
matrix['month'] = matrix['date_block_num'] % 12
matrix['year'] = (matrix['date_block_num'] / 12).astype(np.int8)

## Add month since the last and first sale
The code has been simplified to reduce run time, though still may not be optimal -- ideally we don't need to compute max for each month.

In [ ]:
#Month since last sale for each shop/item pair.
ts = time.time()
last_sale = pd.DataFrame()
for month in range(1,35):    
    last_month = matrix.loc[(matrix['date_block_num']<month)&(matrix['item_cnt_month']>0)].groupby(['item_id','shop_id'])['date_block_num'].max()
    df = pd.DataFrame({'date_block_num':np.ones([last_month.shape[0],])*month,
                       'item_id': last_month.index.get_level_values(0).values,
                       'shop_id': last_month.index.get_level_values(1).values,
                       'item_shop_last_sale': last_month.values})
    last_sale = last_sale.append(df)
last_sale['date_block_num'] = last_sale['date_block_num'].astype(np.int8)

matrix = matrix.merge(last_sale, on=['date_block_num','item_id','shop_id'], how='left')
time.time() - ts

In [ ]:
#Month since last sale for each item.
ts = time.time()
last_sale = pd.DataFrame()
for month in range(1,35):    
    last_month = matrix.loc[(matrix['date_block_num']<month)&(matrix['item_cnt_month']>0)].groupby('item_id')['date_block_num'].max()
    df = pd.DataFrame({'date_block_num':np.ones([last_month.shape[0],])*month,
                       'item_id': last_month.index.values,
                       'item_last_sale': last_month.values})
    last_sale = last_sale.append(df)
last_sale['date_block_num'] = last_sale['date_block_num'].astype(np.int8)

matrix = matrix.merge(last_sale, on=['date_block_num','item_id'], how='left')
time.time() - ts

In [ ]:
# Months since the first sale for each shop/item pair and for item only.
ts = time.time()
matrix['item_shop_first_sale'] = matrix['date_block_num'] - matrix.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
matrix['item_first_sale'] = matrix['date_block_num'] - matrix.groupby('item_id')['date_block_num'].transform('min')
time.time() - ts

## Final preparations
Because of the using 12 as lag value drop first 12 months. Also drop all the columns with this month calculated values (other words which can not be calcucated for the test set).
Lightgbm and XGBboost can deal with missing values, so we will leave the NaNs as it is. 

In [ ]:
matrix = matrix[matrix.date_block_num > 11]
matrix.columns

In [ ]:
matrix.to_pickle('data.pkl')
del matrix
del group
del items
del shops
del cats
del train
# leave test for submission
gc.collect();

# Read data from file

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)

from itertools import product
from sklearn.preprocessing import LabelEncoder

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import time
import sys
import gc
import pickle
sys.version_info

In [ ]:
data = pd.read_pickle('../input/predict-future-sales-ensemble/data.pkl')
data.head()

In [ ]:
data = data[[
    'date_block_num',
    'shop_id',
    #'item_id',
    'item_cnt_month',
    'city_code',
    'item_category_id',
    'type_code','subtype_code',
    'item_cnt_month_lag_1','item_cnt_month_lag_2','item_cnt_month_lag_3','item_cnt_month_lag_6','item_cnt_month_lag_12',
    'item_avg_sale_last_6', 'item_std_sale_last_6',
    'item_avg_sale_last_12', 'item_std_sale_last_12',
    'shop_avg_sale_last_6', 'shop_std_sale_last_6',
    'shop_avg_sale_last_12', 'shop_std_sale_last_12',
    'category_avg_sale_last_12', 'category_std_sale_last_12',
    'city_avg_sale_last_12', 'city_std_sale_last_12',
    'type_avg_sale_last_12', 'type_std_sale_last_12',
    'subtype_avg_sale_last_12', 'subtype_std_sale_last_12',
    'date_avg_item_cnt_lag_1',
    'date_item_avg_item_cnt_lag_1','date_item_avg_item_cnt_lag_2','date_item_avg_item_cnt_lag_3','date_item_avg_item_cnt_lag_6','date_item_avg_item_cnt_lag_12',
    'date_shop_avg_item_cnt_lag_1','date_shop_avg_item_cnt_lag_2','date_shop_avg_item_cnt_lag_3','date_shop_avg_item_cnt_lag_6','date_shop_avg_item_cnt_lag_12',
    'date_cat_avg_item_cnt_lag_1',
    'date_shop_cat_avg_item_cnt_lag_1',
    'date_city_avg_item_cnt_lag_1',
    'date_item_city_avg_item_cnt_lag_1',
    'delta_price_lag',
    'month','year',
    'item_shop_last_sale','item_last_sale',
    'item_shop_first_sale','item_first_sale',
]]

cat_feats = ['shop_id','city_code','item_category_id','type_code','subtype_code']
data.fillna(0,inplace=True)

# **Set up validation strategy**

Validation strategy is 34 month for the test set, 33 month for the validation set and 13-32 months for the train.

In [ ]:
X_train = data[data.date_block_num < 33].drop(['item_cnt_month'], axis=1)
y_train = data[data.date_block_num < 33]['item_cnt_month']
X_valid = data[data.date_block_num == 33].drop(['item_cnt_month'], axis=1)
y_valid = data[data.date_block_num == 33]['item_cnt_month']
X_test = data[data.date_block_num == 34].drop(['item_cnt_month'], axis=1)

del data
gc.collect();


# Models

# Linear Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression
Lr = LinearRegression()
Lr.fit(X_train,y_train)

In [ ]:
Lr.score(X_train,y_train)

In [ ]:
#Predict training set:
train_predictions = Lr.predict(X_train)
valid_predictions = Lr.predict(X_valid)
test_predictions = Lr.predict(X_test).clip(0, 20)

print("Train Root Mean Square Error : %.4g" % metrics.mean_squared_error(y_train, train_predictions))
print("Train R^2 Score (Train): %f" % metrics.r2_score(y_train, train_predictions))

print("Test Root Mean Square Error : %.4g" % metrics.mean_squared_error(y_valid, valid_predictions))
print("Test R^2 Score (Train): %f" % metrics.r2_score(y_valid, valid_predictions))

In [ ]:
Linearregression = pd.DataFrame({'ID':X_test.index,'item_cnt_month':test_predictions})
Linearregression.to_csv('./Linearregression.csv', index= False)

# Random Forest Regressor model - Public Score

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

In [ ]:
RFR = RandomForestRegressor(n_estimators=3,bootstrap=0.8,max_depth=10,
                            min_samples_split=3,max_features=1)
RFR.fit(X_train, y_train)

In [ ]:
cross_val_score(RFR,X_train, y_train, cv=5)

In [ ]:
RFR_prediction = RFR.predict(X_test).clip(0,20)

In [ ]:
#Predict training set:
train_predictions = RFR.predict(X_train)
valid_predictions = RFR.predict(X_valid)
test_predictions = RFR.predict(X_test).clip(0, 20)

print("Train Root Mean Square Error : %.4g" % metrics.mean_squared_error(y_train, train_predictions))
print("Train R^2 Score (Train): %f" % metrics.r2_score(y_train, train_predictions))

print("Test Root Mean Square Error : %.4g" % metrics.mean_squared_error(y_valid, valid_predictions))
print("Test R^2 Score (Train): %f" % metrics.r2_score(y_valid, valid_predictions))

In [ ]:
submissionRF = pd.DataFrame({'ID':X_test.index,'item_cnt_month':test_predictions})
submissionRF.to_csv('./RandomForest.csv', index= False)

# CatBoost Regression Model - 0.94059 Public Score

In [ ]:
%%time
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

model_catboost = CatBoostRegressor(verbose=False, random_seed=566)


model_catboost.fit(X_train,y_train);
prediction_catboost = model_catboost.predict(X_valid)
print(mean_squared_error(y_valid, prediction_catboost, squared=False))

np.save('prediction_catboost.npy', prediction_catboost)
pickle.dump(model_catboost, open('model_catboost.pkl', 'wb'))


In [ ]:
#Predict training set:
train_predictions = model_catboost.predict(X_train)
valid_predictions = model_catboost.predict(X_valid)

print("Train Root Mean Square Error : %.4g" % metrics.mean_squared_error(y_train, train_predictions))
print("Train R^2 Score (Train): %f" % metrics.r2_score(y_train, train_predictions))

print("Test Root Mean Square Error : %.4g" % metrics.mean_squared_error(y_valid, valid_predictions))
print("Test R^2 Score (Train): %f" % metrics.r2_score(y_valid, valid_predictions))

In [ ]:
prediction = model_catboost.predict(X_test)
submission = pd.DataFrame({ "ID": np.arange(prediction.shape[0]),  "item_cnt_month": prediction.clip(0, 20)})
submission.to_csv('catboost.csv', index=False)

### Identifying the most important features using CatBoost Regressor Model

In [ ]:
importance = pd.DataFrame({'feature':X_train.columns, 'importance': model_catboost.feature_importances_})
importance.sort_values('importance', ascending=False).set_index('feature').plot(kind='barh', figsize=(10,20))
plt.show()

In [ ]:
importance.sort_values('importance', ascending=False).feature.values

### Train using the most important features

In [ ]:
%%time
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

main_features = ['item_cnt_month_lag_1', 'item_first_sale',
       'date_item_avg_item_cnt_lag_1', 'item_category_id']



model_catboost = CatBoostRegressor(verbose=False, random_seed=566)


model_catboost.fit(X_train[main_features],y_train);

In [ ]:
predictionimp = model_catboost.predict(X_train[main_features])
predictionimpv = model_catboost.predict(X_valid[main_features])
print('Train important features rmse:', np.sqrt(mean_squared_error(y_train, predictionimp)))
print('Validation important features rmse:', np.sqrt(mean_squared_error(y_valid, predictionimpv)))

## LightGBM - 0.90429 Public Score

In [ ]:
%%time

from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error


#model is fine tuned with all these hyper parameters
model_lgbm = LGBMRegressor(
    max_depth = 8,
    n_estimators = 500,
    colsample_bytree=0.7,
    min_child_weight = 300,
    reg_alpha = 0.1,
    reg_lambda = 1,
    random_state = 42,
)

model_lgbm.fit(
    X_train, 
    y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, y_train), (X_valid, y_valid)], 
    verbose=10, 
    early_stopping_rounds = 40,
    categorical_feature = cat_feats) 

prediction_lgbm = model_lgbm.predict(X_valid)
np.save('prediction_lgbm.npy', prediction_lgbm)
pickle.dump(model_lgbm, open('model_lgbm.pkl', 'wb'))

In [ ]:
prediction_lgbm = model_lgbm.predict(X_valid)
np.save('prediction_lgbm.npy', prediction_lgbm)
pickle.dump(model_lgbm, open('model_lgbm.pkl', 'wb'))

In [ ]:
#Predict training set:
train_predictions = model_lgbm.predict(X_train)
valid_predictions = model_lgbm.predict(X_valid)

print("Train Root Mean Square Error : %.4g" % metrics.mean_squared_error(y_train, train_predictions))
print("Train R^2 Score (Train): %f" % metrics.r2_score(y_train, train_predictions))

print("Test Root Mean Square Error : %.4g" % metrics.mean_squared_error(y_valid, valid_predictions))
print("Test R^2 Score (Train): %f" % metrics.r2_score(y_valid, valid_predictions))

In [ ]:
prediction = model_lgbm.predict(X_test)

submission = pd.DataFrame({
    "ID": np.arange(prediction.shape[0]), 
    "item_cnt_month": prediction.clip(0, 20)
})
submission.to_csv('submission_lgbm.csv', index=False)

# XGBoost - 0.91555 Public Score

In [ ]:
%%time
from xgboost import XGBRegressor

model_xgb = XGBRegressor(max_depth=7,n_estimators=1000,min_child_weight=300,   
                         colsample_bytree=0.8, subsample=0.8, gamma = 0.005,eta=0.1,    
                         seed=42)

model_xgb.fit(X_train, y_train, eval_metric="rmse",
              eval_set=[(X_train, y_train), (X_valid, y_valid)], 
              verbose=10, 
            early_stopping_rounds = 40,
    )


prediction_xgb = model_xgb.predict(X_valid)
prediction_txgb= model_catboost.predict(X_train)
print('Train rmse:', np.sqrt(mean_squared_error(y_train, prediction_tlgbm)))
print('Validation rmse:', np.sqrt(mean_squared_error(y_valid, prediction_lgbm)))
print(mean_squared_error(y_valid, prediction_xgb, squared=False))

np.save('prediction_xgb.npy', prediction_xgb)
pickle.dump(model_xgb, open('model_xgb.pkl', 'wb'))

In [ ]:
prediction = model_xgb.predict(X_test)
#prediction_xgb = prediction
submission = pd.DataFrame({ "ID": np.arange(prediction.shape[0]),     "item_cnt_month": prediction.clip(0, 20) })
submission.to_csv('submission_xgb.csv', index=False)

# Ensemble based on different models

## Meta features

#### Load models,

In [ ]:
model_catboost = pickle.load(open('./model_catboost.pkl','rb'))
model_lgbm = pickle.load(open('./model_lgbm.pkl','rb'))
model_xgb = pickle.load(open('../input/predict-future-sales-ensemble/model_xgb.pkl','rb'))

## Two models - Public score: 0.90462 with LGBM + XGB 
##            - Public score: 0.90617 with LGBM + CatBoost

In [ ]:
from sklearn.metrics import mean_squared_error

# choose models 
# model_index = [0,1]  
model_index = [0,-1]  

alphas_to_try = np.linspace(0, 1, 1001)
rmse_scores = np.empty_like(alphas_to_try)

for ind, alpha in enumerate(alphas_to_try):
    simple_mix_pred = alpha * meta_X_train[:,model_index[0]] + (1 - alpha) * meta_X_train[:,model_index[1]]
    rmse_scores[ind] = mean_squared_error(meta_y_train, simple_mix_pred, squared=False)


best_alpha = alphas_to_try[np.argmin(rmse_scores)]
rmse_train_simple_mix = np.min(rmse_scores) 

print('Best alpha: %f; Corresponding rmse score on train: %f' % (best_alpha, rmse_train_simple_mix))

### Use the best $\alpha$ to make predictions

In [ ]:
meta_prediction = best_alpha * meta_X_test[:,model_index[0]] + (1 - best_alpha) * meta_X_test[:,model_index[1]]
submission = pd.DataFrame({
    "ID": np.arange(meta_prediction.shape[0]), 
    "item_cnt_month": meta_prediction.clip(0, 20).flatten()
 })
submission.to_csv('submission_meta_simple_mix_lgbm_xgb.csv', index=False)

## Three models - Public score: 0.90682

In [ ]:
%%time
from sklearn.metrics import mean_squared_error


alphas_to_try = np.linspace(0, 1, 51)
betas_to_try = np.linspace(0, 1, 51)
gammas_to_try = np.linspace(0, 1, 51)
rmse_scores = np.zeros((len(alphas_to_try), len(betas_to_try),len(gammas_to_try)))

for ind1, alpha in enumerate(alphas_to_try):
    for ind2, beta in enumerate(betas_to_try):  
        for ind3, gamma in enumerate(gammas_to_try):  
            simple_mix_pred = alpha * meta_X_train[:,0] + beta * meta_X_train[:,1] +  gamma * meta_X_train[:,3]
            rmse_scores[ind1, ind2, ind3] = mean_squared_error(meta_y_train, simple_mix_pred, squared=False)


ind1, ind2, ind3 = np.unravel_index(rmse_scores.argmin(), rmse_scores.shape)
best_alpha, best_beta, best_gamma = alphas_to_try[ind1], betas_to_try[ind2], gammas_to_try[ind3]
rmse_train_simple_mix = np.min(rmse_scores) 

print('Best alpha: %f; best beta: %f; best gamma: %f; Corresponding rmse score on train: %f' % (best_alpha, best_beta, best_gamma, rmse_train_simple_mix))

### Use the best $\alpha$, $\beta$ and $\gamma$ to make predictions

In [ ]:

meta_prediction = best_alpha * meta_X_test[:,0] + best_beta * meta_X_test[:,1] + best_gamma * meta_X_test[:,3]
submission = pd.DataFrame({    "ID": np.arange(meta_prediction.shape[0]),  "item_cnt_month": meta_prediction.clip(0, 20).flatten()})
submission.to_csv('submission_meta_simple_mix_3_models.csv', index=False)

## Stacking - Public score: meta_model LR 0.90443

In [ ]:
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor

meta_model = LinearRegression()

#meta_model =LGBMRegressor(
#     max_depth = 2,
#     n_estimators = 500,
#     colsample_bytree=0.7,
#     min_child_weight = 300,
#     reg_alpha = 0.1,
#     reg_lambda = 1,
#     random_state = 42,
# )


meta_model.fit(meta_X_train, meta_y_train)

meta_prediction = meta_model.predict(meta_X_test)
print(mean_squared_error(meta_y_train, meta_model.predict(meta_X_train), squared=False))

# np.save('meta_prediction_lr_2.npy', meta_prediction)

In [ ]:
submission = pd.DataFrame({
    "ID": np.arange(meta_prediction.shape[0]), 
     "item_cnt_month": meta_prediction.clip(0, 20).flatten()
 })
submission.to_csv('Stacking.csv', index=False)